<a href="https://colab.research.google.com/github/epsilon-deltta/sl-for-dl/blob/master/dl-torch/baseline_fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader as dl

from torchvision import datasets as dt
from torchvision.transforms import ToTensor,Lambda , Compose
import matplotlib.pyplot as plt

tr = dt.FashionMNIST(root='fa',download=True,train=True,transform=ToTensor() )
te = dt.FashionMNIST(root='fa',download=True,train=False,transform=ToTensor() )
trdl = dl(tr,256)
tedl = dl(te,256)

class cons(nn.Module):
    def __init__(self):
        super(cons,self).__init__()
        self.front = nn.Sequential(
            nn.Conv2d(1,32,3),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            nn.ReLU() ,#batchnorm 2d ?
            nn.Conv2d(32,16,2),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Flatten()
        )
        sple = torch.rand(1,1,28,28)
        front_out = self.front(sple).size()[-1]
        self.body = nn.Sequential(
            nn.Linear(front_out,256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(p=0.2),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            # nn.Linear(128,10),
        )
        self.fc = nn.Sequential(
            nn.Linear(128,10),
            nn.Softmax(dim=1)
        )
    def forward(self,x):
        x = self.front(x)
        x = self.body(x)
        x = self.fc(x)
        return x
device = 'cuda' if torch.cuda.is_available() else 'cpu'
con0 = cons().to(device)
loss = nn.CrossEntropyLoss()
opt  = torch.optim.Adam(con0.parameters() )

def train(dl,model,lossf,opt):
    for x,y in dl:
        x,y = x.to(device),y.to(device)
        pre = model(x)
        loss = lossf(pre,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

def test(dl,model,lossf):
    model.eval()
    size, acc , losses = len(dl.dataset) ,0,0
    with torch.no_grad():
        for x,y in dl:
            x,y = x.to(device),y.to(device)
            pre = model(x)
            loss = lossf(pre,y)

            acc += (pre.argmax(1)==y).type(torch.float).sum().item()
            losses += loss.item()
    print(f'{acc/size} : {losses/size}')
for _ in range(12):
    train(trdl,con0,loss,opt)
    test(tedl,con0,loss)

torch.save(con0,'con2l3v1.pth')
model = torch.load('con2l3v1.pth')

model = torch.load('con2l3v1.pth')
model.eval()
with torch.no_grad():
    classes = dt.FashionMNIST.classes
    for i in range(10):
        x,y = te[i][0],te[i][1]
        pre = model(x.unsqueeze(0))

        print(f'{classes[y]} : {classes[pre.argmax(1)]}')